<a href="https://colab.research.google.com/github/KGYAMFI22/csv-leaflet-map/blob/main/Change_Detection_for_Wildfire_Burn_Severity_Using_Sentinel_1%2C_Sentinel_2%2C_and_GOES_16_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



🔬 Dataset Description:
We utilize Sentinel-1, Sentinel-2, and GOES-16 satellite imagery for wildfire analysis.

Dataset	Source	Usage
Sentinel-2 (Optical)	Copernicus Hub	Compute NBR & dNBR for burn severity
Sentinel-1 (SAR)	Copernicus Hub	Analyze backscatter changes to detect burned areas
GOES-16	NOAA	Real-time fire monitoring
🔥 Study Area: California Wildfire (June–Sept 2023)




Step 1: Authenticate Google Earth Engine

In [ ]:
# Install necessary libraries
!pip install earthengine-api geemap folium --quiet

import ee
import geemap
import folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.5 MB/s eta 0:00:00


In [ ]:
# Authenticate with Google Earth Engine
ee.Authenticate()
# Initialize Earth Engine with your project ID
ee.Initialize(project='ee-kgyamfi')

Step 2: Load Pre- and Post-Fire Satellite Imagery

In [ ]:
region = ee.Geometry.Point([-120.74, 38.79])  # Example wildfire site

# Load Sentinel-2 data
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(region).filterDate('2023-06-01', '2023-09-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).median()

# Load Sentinel-1 SAR data
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(region).filterDate('2023-06-01', '2023-09-01') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')).median()


Step 3: Compute Burn Indices (NBR & dNBR)

In [ ]:
def calc_nbr(image):
    nir = image.select('B8')
    swir = image.select('B12')
    nbr = nir.subtract(swir).divide(nir.add(swir)).rename('NBR')
    return image.addBands(nbr)

# Compute pre- and post-fire NBR
pre_fire_nbr = calc_nbr(sentinel2).select('NBR')
post_fire_nbr = calc_nbr(sentinel2).select('NBR')

# Compute dNBR (change in burn index)
dNBR = pre_fire_nbr.subtract(post_fire_nbr).rename('dNBR')

# Burn severity classification
severity = dNBR.expression(
    "(b(0) > 0.66) ? 4 : (b(0) > 0.44) ? 3 : (b(0) > 0.22) ? 2 : (b(0) > 0.1) ? 1 : 0"
).rename('BurnSeverity')


Step 4: Detect Burned Areas with SAR Change Detection

In [ ]:
# Compute change in SAR backscatter (VV band)
pre_fire_sar = sentinel1.select('VV')
post_fire_sar = sentinel1.select('VV')
sar_change = pre_fire_sar.subtract(post_fire_sar).rename('SAR_Change')

# Threshold SAR change to detect burned areas
burned_sar = sar_change.lt(-2).rename('Burned_Area')


Step 5: Visualizing Burn Severity Maps

In [ ]:
import folium
from geemap import foliumap

Map = geemap.Map(center=[38.79, -120.74], zoom=8)
Map.addLayer(dNBR, {"min": -1, "max": 1, "palette": ['green', 'yellow', 'red']}, "Burn Severity (dNBR)")
Map.addLayer(burned_sar, {"min": 0, "max": 1, "palette": ['black', 'red']}, "SAR Burned Areas")
Map.add_child(folium.LayerControl())
Map
